In [1]:
import sqlalchemy as sqlal
import numpy as np
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import calendar
sns.set_style('whitegrid')
%matplotlib inline

In [2]:
def extract_from_zillow_by_city (city,state):
    if any(city in s for s in zillow_by_city['RegionName']):
        ind = zillow_by_city.index[(zillow_by_city['RegionName']==city) & (zillow_by_city['State']==state)]
        city_df = zillow_by_city.loc[ind,:]
        city_df = city_df.T
        city_df = city_df.drop(city_df.index[0:7])
        city_df.columns = [city]
        return city_df
    else:
        return "Does not exist in database."

In [3]:
def calculate_moving_average (df):
    MA_day = [14,30,60]
    for MA in MA_day:
        column_name = "MA for %s days" %(str(MA))
        df[column_name] = df['Listing'].rolling(MA).mean()
    return df

Establish connection and pull data from database

In [4]:
username = ""
password = ""
host = ""
database = ""
engine = sqlal.create_engine("mysql://" + username + ":" + password + "@" + host + "/" + database)
con = engine.connect()
data = "zillow_by_city"
zillow_by_city = pd.read_sql('SELECT * FROM %s' %(data), con=con)
con.close()

OperationalError: (MySQLdb._exceptions.OperationalError) (1049, "Unknown database 'housing'") (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
zillow_by_city.head()

Extract cities from zillow_by_city to create a single dataframe

In [ ]:
cities = ['Stockton', 'New York', 'San Jose']
states = ['CA', 'NY', 'CA']
usa = pd.DataFrame()
for i in range(0,len(cities)):
    dummy_city = extract_from_zillow_by_city(cities[i],states[i]) 
    usa = pd.concat([usa, dummy_city], axis=1)
usa.columns = cities    
usa.plot()

Analyze a single city

In [ ]:
city = 'New York' #Enter the city that you want to analyze
usa[city].plot(legend=True, figsize=(10,5), color='r')
plt.ylabel("US dollar")

In [ ]:
city_copy = usa[city].copy()

In [ ]:
df = pd.DataFrame(city_copy)
df.rename({city:'Listing'}, axis='columns', inplace=True)

In [ ]:
df = calculate_moving_average(df)

In [ ]:
df.plot(legend=True, figsize=(10,5))
plt.title('Moving Average of %s Listing Price' %(city), loc='center') 

In [ ]:
df['percent'] = df['Listing'].pct_change().multiply(100)

In [ ]:
df['percent'].plot(legend=True, figsize=(10,5), color='g' )
plt.ylabel('percent change')

In [ ]:
sns.distplot(df['percent'].dropna(), bins=50, color='r')
plt.xlabel('Monthly Percent Change')

In [ ]:
df['percent'].mean()